In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

from vocab import VocabEntry
from utils import pad_sents_char

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
sentences = [['Human:', 'What', 'do', 'we', 'want?'], 
             ['Computer:', 'Natural', 'language', 'processing!'],
             ['Human:', 'When', 'do', 'we', 'want', 'it?'], 
             ['Computer:', 'When', 'do', 'we', 'want', 'what?']]

len(sentences)

4

### VocabEntry

In [74]:
from vocab import VocabEntry
voc_entry = VocabEntry()
voc_entry_output = voc_entry.to_input_tensor_char(sentences, 'cpu')
voc_entry_output.shape
#voc_entry_output

torch.Size([6, 4, 21])

### Vocab

In [75]:
from utils import read_corpus
from vocab import Vocab
src_sents = read_corpus('./en_es_data/train_tiny.es', source='src')
tgt_sents = read_corpus('./en_es_data/train_tiny.en', source='tgt')

vocab = Vocab.build(src_sents, tgt_sents, vocab_size=200, freq_cutoff=1)

initialize source vocabulary ..
number of word types: 128, number of word types w/ frequency >= 1: 128
initialize target vocabulary ..
number of word types: 130, number of word types w/ frequency >= 1: 130


In [5]:
vocab.src['<pad>']

0

### CNN

In [26]:
from cnn import CNN

input_data = torch.randn(2,3,6) # num_of_sentence, num_of_dim per word, num_of_words_per_sent

cnn_obj = CNN(3, 1)

output = cnn_obj(input_data)

output.shape
output

torch.Size([2, 1])

tensor([[0.7477],
        [0.8650]], grad_fn=<MaxBackward0>)

### Highway

In [76]:
from highway import Highway
highway_obj = Highway(5)

x_input = torch.randn(2,3,5)

highway_obj(x_input).shape

torch.Size([2, 3, 5])

### Model_Embedding

In [77]:
# initialization of model embedding
from model_embeddings import ModelEmbeddings

voc_entry = VocabEntry()
voc_entry_output = voc_entry.to_input_tensor_char(sentences, 'cpu')

model_embed_obj = ModelEmbeddings(5, vocab.src)

#### x_padded to x_word_emb. that is the goal

- feed x_padded to x_emb
- Embedding to CNN
- CNN to Highway

In [78]:
## use of model embedding

voc_entry = VocabEntry()
voc_entry_output = voc_entry.to_input_tensor_char(sentences, 'cpu')
voc_entry_output.shape
#voc_entry_output

output = model_embed_obj(voc_entry_output)
output.shape

torch.Size([6, 4, 21])

torch.Size([6, 4, 5])

## CharDecoder

In [246]:
from char_decoder import CharDecoder
import json

class DummyVocab():
    def __init__(self):
        self.char2id = json.load(open('./sanity_check_en_es_data/char_vocab_sanity_check.json', 'r'))
        self.id2char = {id: char for char, id in self.char2id.items()}
        self.char_unk = self.char2id['<unk>']
        self.start_of_word = self.char2id["{"]
        self.end_of_word = self.char2id["}"]

char_vocab = DummyVocab()
    
char_decoder = CharDecoder(
        hidden_size=3,
        char_embedding_size=3,
        target_vocab=char_vocab)

In [252]:
a = torch.rand(3,5,10)
a.shape
len(char_vocab.char2id)

a.shape
a.numel()
a.view(-1, len(char_vocab.char2id)).shape
a.view(-1, len(char_vocab.char2id)).numel()

torch.Size([3, 5, 10])

30

In [262]:
inpt = torch.zeros(4, 3, dtype=torch.long)
inpt
inpt[1:].contiguous().view(-1)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:

(s_t.view(-1, len(self.target_vocab.char2id)), char_sequence[1:].contiguous().view(-1))


In [264]:
a = torch.rand(20,2,21)
a.shape

torch.Size([20, 2, 21])

In [269]:
#a[1:].view(-1, 21)
a[1:].shape
a[1:].view(-1, 21).shape

torch.Size([19, 2, 21])

torch.Size([38, 21])

## Embedding Exploration

In [32]:
embedding = nn.Embedding(10, 3) # maximum number of words is 10. Each word to 3 dims
#embedding.weight
input_data = torch.LongTensor([[0,2,0,5, 9]])   # here 0, 2, 5, 9 are the integer index of the word
input_data.shape ## torch.Size([1, 5])

output = embedding(input_data)
output.shape   # torch.Size([1, 5, 3])

torch.Size([1, 5])

torch.Size([1, 5, 3])

In [ ]:
embedding = nn.Embedding(10, 3, padding_idx=0)
input = torch.LongTensor([[0,2,0,5]])
embedding(input)


In [ ]:
embedding = nn.Embedding(10, 3)
embedding.weight
input = torch.LongTensor([[0,2,0,5, 9]])
embedding(input)

In [ ]:
embedding = nn.Embedding(10, 3, padding_idx=2)
embedding.weight
input = torch.LongTensor([[0,2,0,5, 9]])
input
input.shape
output = embedding(input)
output
output.shape

In [28]:
embedding = nn.Embedding(10, 3, padding_idx=0)

input = torch.randint(0,10, (15, 1, 5))  # 15 sent lenght, 1 batch, each senten has 5 words
input.shape
output = embedding(input)
output.shape

torch.Size([15, 1, 5])

torch.Size([15, 1, 5, 3])

In [ ]:
output[1,0,:2]

In [ ]:
input[1,0,:2]

## Conv1D exploration

In [ ]:
my_input = torch.ones(1, 4, 5) # there are 4 words each of which has 5 dimensions
my_input.shape

# we want number of dimensions as rows and number of words as columns
rand_arr_permute = my_input.clone().permute(0,2,1)
rand_arr_permute.shape

# how conv1d input and output shapes are related
conv1 = nn.Conv1d(5, 1, (2,)) # 5 dims (see my_input), 1 output channel, kernel=(2,)
conv1.weight
conv1.bias

with torch.no_grad():
    conv1.weight.fill_(1)
    conv1.bias.fill_(0)
    print(torch.sum((rand_arr_permute[:,:,:2]*conv1.weight)) + conv1.bias)
    
    output = conv1(rand_arr_permute)
    print(output)
    print(output.shape)


#### Conv1D with 4 dimensions

In [55]:
a = torch.randint(1, 20, (2,3,4,10)) # batch 2, sentences 3, words 4 per sent, embed 5 per word
a = torch.randn(2,3,4,10)
a.shape

# let us permute so bring 10 at -2 
b = a.permute(0,1,3,2)
b.shape

# but conv1d does not take 4 dimension. So, reduce it to 3
b_reduced = b.view(-1, b.shape[-2], b.shape[-1])
b_reduced.shape

torch.Size([2, 3, 4, 10])

torch.Size([2, 3, 10, 4])

torch.Size([6, 10, 4])

In [56]:
# goal is to reduce the embed size of 10  to 5
# b_reduced.shape = torch.Size([6, 10, 4])
# expected shape is 6, 5 , <>
conv1 = nn.Conv1d(10, 5, 2) 

In [58]:
c = conv1(b_reduced)
c.shape

torch.Size([6, 5, 3])

In [ ]:
sents = [[[1, 30, 2],         [1, 31, 2], [1, 32, 70, 2]], 
         [[1, 85, 33, 85, 2], [1, 32, 2], [1, 31, 2], [1, 30, 2]]]


In [ ]:
[[[1, 11, 50, 42, 30, 43, 71, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 26, 37, 30, 49, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 33, 44, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 34, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 30, 43, 49, 70, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[1, 6, 44, 42, 45, 50, 49, 34, 47, 71, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 17, 30, 49, 50, 47, 30, 41, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 41, 30, 43, 36, 50, 30, 36, 34, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 45, 47, 44, 32, 34, 48, 48, 38, 43, 36, 69, 2, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[1, 11, 50, 42, 30, 43, 71, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 26, 37, 34, 43, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 33, 44, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 34, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 30, 43, 49, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 38, 49, 70, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[1, 6, 44, 42, 45, 50, 49, 34, 47, 71, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 26, 37, 34, 43, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 33, 44, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 34, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 30, 43, 49, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 37, 30, 49, 70, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]] 

[[[1, 11, 50, 42, 30, 43, 71, 2, 0, 0, 0, 0, 0], [1, 26, 37, 30, 49, 2, 0, 0, 0, 0, 0, 0, 0], [1, 33, 44, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 34, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 30, 43, 49, 70, 2, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[1, 6, 44, 42, 45, 50, 49, 34, 47, 71, 2, 0, 0], [1, 17, 30, 49, 50, 47, 30, 41, 2, 0, 0, 0, 0], [1, 41, 30, 43, 36, 50, 30, 36, 34, 2, 0, 0, 0], [1, 45, 47, 44, 32, 34, 48, 48, 38, 43, 36, 69, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[1, 11, 50, 42, 30, 43, 71, 2, 0, 0, 0, 0, 0], [1, 26, 37, 34, 43, 2, 0, 0, 0, 0, 0, 0, 0], [1, 33, 44, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 34, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 30, 43, 49, 2, 0, 0, 0, 0, 0, 0, 0], [1, 38, 49, 70, 2, 0, 0, 0, 0, 0, 0, 0, 0]], [[1, 6, 44, 42, 45, 50, 49, 34, 47, 71, 2, 0, 0], [1, 26, 37, 34, 43, 2, 0, 0, 0, 0, 0, 0, 0], [1, 33, 44, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 34, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 52, 30, 43, 49, 2, 0, 0, 0, 0, 0, 0, 0], [1, 52, 37, 30, 49, 70, 2, 0, 0, 0, 0, 0, 0]]]

In [ ]:
import torch


In [ ]:
a = [[1,2], [3,4]]
torch.IntTensor(a)

In [ ]:
max_sent_len = max([len(s) for s in sents])
max_word_len = max([max([len(w) for w in s]) for s in sents])
sents_padded = []

char_pad_token = 0

for s in sents:
    words_padded = []
    for w in s:
        padded_word = [char_pad_token] * max_word_len
        padded_word[:len(w)] = w
        words_padded.append(padded_word)
    print(words_padded)
        
    padded_sent = [[char_pad_token]*max_word_len] * max_sent_len
    padded_sent[:len(words_padded)] = words_padded
    sents_padded.append(padded_sent)
    


sents_padded

In [ ]:
char_pad_token = 0
[[char_pad_token]*max_word_len]*max_sent_len

In [ ]:
my_input = torch.ones(1, 4, 5) # there are 3 words each of which has 5 dimensions
my_input.shape

# we want number of dimensions as rows and number of words as columns
rand_arr_permute = my_input.clone().permute(0,2,1)

print('input shape ', rand_arr_permute.shape)

# how conv1d input and output shapes are related
conv1 = nn.Conv1d(5, 50, (2,)) # here 50 is embedding dimension
output = conv1(rand_arr_permute)

print('conv1 output shape', output.shape)

# but, we are getting 3 in torch.Size([1, 50, 3])

#relu

relu = torch.nn.ReLU()
output_relu = relu(output)
print('relu shape ', output_relu.shape)

# maxpool

In [ ]:
# pool of size=3, stride=2

input_data = torch.randn(1, 1, 50)
print('input shape ', input_data.shape)
m = nn.MaxPool1d(1, stride=1)
output = m(input_data)
output.shape


In [ ]:
a = torch.randn(1,1,10)
#a = torch.randint(1,10, (1,1,10))
a.shape
a

m = nn.MaxPool1d(3, stride=1)
m_a = m(a)
m_a.shape
m_a

In [ ]:
D_in = 10

linear1 = torch.nn.Linear(D_in, D_in)
linear2 = torch.nn.Linear(D_in, D_in)
relu = torch.nn.ReLU()
tanh_gate = torch.nn.Tanh()
    


In [ ]:
x_conv_out = torch.randn(40,15,D_in)
print('Input shape ', x_conv_out.shape)

x_proj = relu(linear1(x_conv_out))
x_gate = tanh_gate(linear2(x_conv_out))


x_proj.shape, x_gate.shape

x_highway = (x_proj * x_gate) +  (1-x_gate)*x_conv_out # skip connection

x_highway.shape

In [ ]:
a = torch.randn(2,3)
b = torch.randn(2,3)
c = a * b
print(c.shape)

### Linear layer

In [ ]:
linear_model = nn.Linear(3, 2, bias=False)  # (num of input features, num of output features, bias=True)
print('weights are:',linear_model.weight, '\n\nbias are: ',linear_model.bias)

numel_list = [p.numel() for p in linear_model.parameters()]
print('Total parameters ', sum(numel_list), numel_list)

In [ ]:
m = nn.Linear(20, 30)

input_data = torch.randn(2, 128, 20)
output = m(input_data)

print(output.shape)


In [ ]:
del input

In [ ]:
x = torch.ones(1) # batch is missing
x.shape
linear_model(x)

In [ ]:
a = torch.randn(1,1,10)
#a = torch.randint(1,10, (1,1,10))
a.shape
a

m = nn.MaxPool1d(3, stride=1)
m_a = m(a)
m_a.shape
m_a



In [ ]:
a = torch.randint(1,100, (2,3,4))
a

In [ ]:
b = torch.max(a, dim=2)
type(b)

In [ ]:
b.values.shape
b.values

In [ ]:
b[0].shape

In [ ]:
b.indices

## Pad sequence

- Given variable lenght of the data, we want to pad them to uniform lenght
- to do that, use pad_sequence().

- Now, after padding, we see lot of padded value, a waste of space. 
- So, 

In [94]:
from torch.nn.utils.rnn import pad_sequence
a = torch.randint(1,10,(5, 4))
b = torch.randint(11,20,(4, 4))
c = torch.randint(21,30,(3, 4))
output = pad_sequence([a, b, c], batch_first = True, padding_value=100)
output.shape

torch.Size([3, 5, 4])

In [88]:
a
b
c

tensor([[9, 7, 7, 3],
        [4, 1, 8, 8],
        [9, 8, 4, 6],
        [9, 3, 9, 9],
        [7, 6, 4, 9]])

tensor([[13, 14, 19, 12],
        [17, 14, 14, 17],
        [17, 12, 16, 11],
        [19, 16, 12, 18]])

tensor([[26, 27, 28, 29],
        [29, 28, 25, 25],
        [28, 21, 26, 25]])

In [107]:
output.shape
output

torch.Size([3, 5, 4])

tensor([[[  1,   7,   9,   7],
         [  4,   5,   3,   2],
         [  6,   2,   1,   4],
         [  4,   4,   9,   5],
         [  4,   4,   7,   3]],

        [[ 11,  19,  17,  19],
         [ 17,  15,  19,  18],
         [ 17,  14,  16,  15],
         [ 18,  18,  16,  17],
         [100, 100, 100, 100]],

        [[ 28,  27,  26,  28],
         [ 27,  21,  21,  28],
         [ 23,  29,  23,  25],
         [100, 100, 100, 100],
         [100, 100, 100, 100]]])

#### Compress the data by removing pad_token

In [96]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

new_output = pack_padded_sequence(output, [len(i) for i in [a,b,c]] ,batch_first=True)

In [106]:
new_output

PackedSequence(data=tensor([[ 1,  7,  9,  7],
        [11, 19, 17, 19],
        [28, 27, 26, 28],
        [ 4,  5,  3,  2],
        [17, 15, 19, 18],
        [27, 21, 21, 28],
        [ 6,  2,  1,  4],
        [17, 14, 16, 15],
        [23, 29, 23, 25],
        [ 4,  4,  9,  5],
        [18, 18, 16, 17],
        [ 4,  4,  7,  3]]), batch_sizes=tensor([3, 3, 3, 2, 1]), sorted_indices=None, unsorted_indices=None)

In [108]:
new_output.data

tensor([[ 1,  7,  9,  7],
        [11, 19, 17, 19],
        [28, 27, 26, 28],
        [ 4,  5,  3,  2],
        [17, 15, 19, 18],
        [27, 21, 21, 28],
        [ 6,  2,  1,  4],
        [17, 14, 16, 15],
        [23, 29, 23, 25],
        [ 4,  4,  9,  5],
        [18, 18, 16, 17],
        [ 4,  4,  7,  3]])

In [109]:
new_output.batch_sizes

tensor([3, 3, 3, 2, 1])

#### Unpacked the compressed packed data

In [110]:
seq_unpacked, lens_unpacked = pad_packed_sequence(new_output, batch_first=True)

In [112]:
lens_unpacked

tensor([5, 4, 3])

In [111]:
seq_unpacked

tensor([[[ 1,  7,  9,  7],
         [ 4,  5,  3,  2],
         [ 6,  2,  1,  4],
         [ 4,  4,  9,  5],
         [ 4,  4,  7,  3]],

        [[11, 19, 17, 19],
         [17, 15, 19, 18],
         [17, 14, 16, 15],
         [18, 18, 16, 17],
         [ 0,  0,  0,  0]],

        [[28, 27, 26, 28],
         [27, 21, 21, 28],
         [23, 29, 23, 25],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0]]])

### Log likelihood  entropy

In [241]:
# this is when logsoftmax and loss (which is either sum or mean of the 'target' element)  


input_data = torch.randn(3, 5, requires_grad=True)

input_data
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
target

softmax = nn.LogSoftmax(dim=1)
a = softmax(input_data)

print('a is the softmax of each row ', a)

print()
print('loss when sum is used ', -sum([a[i,j.item()] for i,j in enumerate(target)]))
print('loss when mean is used ', -sum([a[i,j.item()] for i,j in enumerate(target)])/len(target))

loss = nn.NLLLoss(reduction='mean')
output = loss(softmax(input_data), target)
output

print("loss using  cross entropy ")
cross_entropy_loss = nn.CrossEntropyLoss()
cross_entropy_loss(input_data, target)



tensor([[-0.9388,  1.0749,  2.3506,  1.8561,  1.0047],
        [ 0.4089,  0.3882,  0.4298, -1.2429, -0.1434],
        [ 1.1741, -0.7568, -0.0546, -0.0075, -0.2411]], requires_grad=True)

tensor([1, 0, 4])

a is the softmax of each row  tensor([[-4.0718, -2.0581, -0.7824, -1.2769, -2.1282],
        [-1.3265, -1.3472, -1.3057, -2.9784, -1.8788],
        [-0.6868, -2.6177, -1.9155, -1.8684, -2.1020]],
       grad_fn=<LogSoftmaxBackward>)

loss when sum is used  tensor(5.4866, grad_fn=<NegBackward>)
loss when mean is used  tensor(1.8289, grad_fn=<DivBackward0>)


tensor(1.8289, grad_fn=<NllLossBackward>)

loss using  cross entropy 


tensor(1.8289, grad_fn=<NllLossBackward>)

tensor(2.2538, grad_fn=<NllLossBackward>)

In [225]:
>>> loss = nn.CrossEntropyLoss()
>>> input = torch.randn(3, 5, requires_grad=True)
>>> target = torch.empty(3, dtype=torch.long).random_(5)
>>> output = loss(input, target)


In [161]:
rows = 3
columns = 5

[[np.exp(col+(columns*row))  for col in range(columns)] for row in range(rows)]

[[1.0,
  2.718281828459045,
  7.38905609893065,
  20.085536923187668,
  54.598150033144236],
 [148.4131591025766,
  403.4287934927351,
  1096.6331584284585,
  2980.9579870417283,
  8103.083927575384],
 [22026.465794806718,
  59874.14171519782,
  162754.79141900392,
  442413.3920089205,
  1202604.2841647768]]